# Yolo en détail

Dans ce notebook, nous allons analyser en détails le fonctionnement du modèle [YOLO](https://arxiv.org/pdf/1506.02640).

## Détection d'objets

### Histoire de la détection d'objets

Peu après le fameux [papier](https://proceedings.neurips.cc/paper_files/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf) ayant propulsé le deep learning sur le devant de la scène. Les chercheurs en traitement d'images du monde entier ont commencé à travailler sur des modèles de Deep Learning.  
 
Comme indiqué dans le notebook précédent, il y a trois principales catégories d'algorithme de traitement d'images : la classication, la détection et la segmentation.   

Côté classification, l'approche est assez directe dès lors qu'on a les ressources pour implémenter un modèle de deep learning profond. Par contre, pour la détection, il faut être plus inventif.  

En 2014, un groupe de chercheur propose le papier [Rich feature hierarchies for accurate object detection and semantic segmentation](https://arxiv.org/pdf/1311.2524) plus connu sous le nom R-CNN. Ce papier, qui a eu une grande influence, introduit une architecture en deux étapes pour la détection d'objets et offre des performances remarquables. Le principal problème de cette approche est son temps de traîtement trop lent qui ne permet pas de faire de la détection en temps réel. De nombreuses méthodes ont essayé de résoudre ce problème de temps en proposant des architectures différentes comme [fast R-CNN](https://arxiv.org/pdf/1504.08083), [faster R-CNN](https://arxiv.org/pdf/1506.01497) et [mask R-CNN](https://arxiv.org/pdf/1703.06870). Ces méthodes améliorent grandement le R-CNN de base mais ne sont pas suffisantes pour du temps réel dans la plupart des cas. 

En 2015, un article va provoquer un chamboulement majeur dans le domaine de la détection d'objets. Ce papier est [You Only Look Once: Unified, Real-Time Object Detection](https://arxiv.org/pdf/1506.02640).

### You Only Look Once 

Les approches précédentes se basaient sur une proposition de régions suivie d'une classification. Il s'agissait donc d'un façon d'utiliser les puissants classifieurs dans une tâche de détection d'objets.  

Le papier You Only Look Once (YOLO) propose de prédire les bounding box et les probabilités d'appartenance à une classe directement grâce à un unique réseau de neurones. Cette architecture est beaucoup plus rapide et permet d'atteindre des vitesses de traitement allant jusqu'à 45 images par seconde.   

C'est une révolution dans le domaine de la détection d'objets !  

Mais alors, comment ça marche ? 

## YOLO : Comment ça marche ? 

Cette partie décrit l'architecture de YOLO en s'inspirant du [blogpost](https://medium.com/analytics-vidhya/yolo-explained-5b6f4564f31) qui je vous invite à consulter. Les images sont tirées de ce blogpost ou du papier original.

### Séparation en grille

Le principe de base de YOLO est de diviser l'image en plus petites images à l'aide d'une grille de dimension $S \times S$ de cette manière :  

<img src="images/grid.png" alt="yolo" width="300"/> 


La cellule contenant le centre d'un objet (par exemple le chien ou le vélo) est la cellule responsable de la détection de cet objet (pour le calcul du loss). Chaque cellule de la grille va prédire $B$ bounding boxes (paramètrable, 2 sur le papier original) et un score de confiance pour chacune des bounding box prédites. La bounding box prédite contient les valeurs $x,y,w,h,c$ ou $(x,y)$ sont la position du centre dans la grille, $(w,h)$ sont la dimension de la box en pourcentage de l'image entière et $c$ est la confiance du modèle (probabilité).

Pour calculer la précision de notre bounding box lors de l'entraînement (composante du loss), on utilise l'intersection over union qui est définie comme :   
$\frac{pred_{box}\cap label_{box}}{pred_{box} \cup label_{box}}$

<img src="images/iou.png" alt="iou" width="300"/> 


En plus de prédire la bounding box et la confiance, chaque cellule va aussi prédire la classe de l'objet. Cette classe est representée par un vecteur one_hot (qui contient uniquement des 0 sauf un 1 dans la bonne classe) dans les annotations. Il est important de préciser que chaque cellule peut prédire plusieurs bounding box mais une seule classe. C'est une des limitations de l'algorithme : si il y a plusieurs objets dans la même cellule, le modèle ne pourra pas prédire correctement chaque objet.   

Maitenant qu'on a toutes les informations, on peut calculer la dimension de sortie du réseau. On a $S \times S$ cellules, chaque cellule va prédire $B$ bouding box et $C$ probabilité (avec $C$ le nombre de classe).   
La prédiction du modèle est donc de taille : $S \times S \times (C +B \times 5) $

Cela nous amène à la figure suivante :  

<img src="images/yolo.png" alt="yolo" width="500"/> 

La figure du centre en haut represente les bounding box prédites par le modèle (celles avec un trait plus gros sont les scores de confiance élévés) et la figure du centre en bas représente la classe prédite dans chaque cellule (en bleu la classe chien, en jaune la classe vélo et en rose la classe voiture).

### Architecture du modèle 

L'architecture du modèle YOLO en terme d'agencement des couches est aussi assez particulière. Il y a 3 composants principals : head, neck and backbone.  
- **Backbone** : C'est la partie la plus importante du réseau qui est consistuée d'une série de convolutions pour détecter les features les plus importantes. Typiquement, cette partie est souvent pré-entraîné sur un dataset de classification.
- **Neck and Head** : Ces couches sont responsables du traitement de l'output des couches de convolutions pour sortir une prédiction de taille $S \times S \times (C +B \times 5)$

Dans le papier original de YOLO, la grille est de taille 7x7, il y a 20 classes ([Pascal VOC](http://host.robots.ox.ac.uk/pascal/VOC/)) et on prédit deux bounding box par cellule. Cela nous donne une prédiction de taille   
$7 \times 7 \times (20 +2 \times 5) = 1470$

### Entraînement du modèle 

Les valeurs d'entraînement du modèle (taille d'image, epochs, nombres de couches, batch_size etc ...) sont détaillés dans le papier original et nous n'allons pas entrer dans les détails ici.   

Par contre, il est intéressant de s'attarder un peu sur la fonction de loss. L'idée logique de base serait d'utiliser simplement le loss MSE entre nos prédictions et les labels. Cependant, ça ne fonctionne pas tel quel car le modèle donnerait une importance similaire à la qualité de la localisation et à la veracité de la prédiction. En pratique, on utilise une pondération sur les loss $\lambda_{coord}$ et $\lambda_{noobj}$. Les valeurs du papier original sont défini à 5 pour $\lambda_{coord}$ et 0.5 pour $\lambda_{noobj}$. A noter que $\lambda_{noobj}$ est utilisé uniquement sur les cellules où il n'y a pas d'objets pour éviter que son score de confiance d'une valeur proche de 0 puisse trop impacter les cellules contenant des objets. 

### Limitations de YOLO

Nous avons déjà évoqué sa principale limitation qui est de ne prédire qu'un nombre limité de bounding box par cellule et de ne pas permettre la détection d'objets de différentes catégories dans la même cellule. Cela pose problème lorsque l'on veut détecter des personnes dans une foule par exemple. 

## Amélioration de YOLO

Nous avons vu que YOLO est un modèle très performant et rapide pour la détection d'objets dans les images. C'est pourquoi, de nombreux chercheurs ont cherché à l'améliorer en proposant diverses optimisations. Aujourd'hui encore, de nouvelles versions de Yolo sortent régulierement.   

Cette partie présente chronologiquement les différentes version de YOLO.  

### YOLOv2 (2017) - aussi connu sous le nom de YOLO9000
**Papier : [YOLO9000: Better, Faster, Stronger](https://arxiv.org/pdf/1612.08242)**

**Innovations :**
  - Introduction de l'idée d'ancres (anchors) pour améliorer la précision des prédictions de boîte.
  - Passer de la résolution d'entrée de 224x224 à 416x416 pour améliorer la détection d'objets de petite taille.

### YOLOv3 (2018)
**Papier : [YOLOv3: An Incremental Improvement](https://arxiv.org/abs/1804.02767)**   

**Innovations :**
  - Utilisation d'un modèle plus profond avec une architecture Darknet-53, un réseau de neurones convolutifs résiduel.
  - Détection multi-échelle, avec des prédictions faites à trois niveaux de granularité différents (feature maps de différentes tailles).

### YOLOv4 (2020)
**Papier : [YOLOv4: Optimal Speed and Accuracy of Object Detection](https://arxiv.org/abs/2004.10934)**

**Innovations :**
  - Utilisation de la Backbone CSPDarknet53 pour une meilleure performance.
  - Améliorations des têtes de détection avec PANet (Path Aggregation Network) pour améliorer les flux d'informations.
  - Introduction du concept de Mosaic Data Augmentation pour enrichir la diversité des données d'entraînement.
  - Ajout de diverses techniques modernes comme DropBlock, Mish activation, et SPP (Spatial Pyramid Pooling).

### YOLOv5 (2020) - Développé par Ultralytics

**Innovations :**
  - Pas de papier officiel, mais des améliorations pratiques dans l'implémentation et la performance.
  - Modèle plus léger et plus facile à entraîner avec une meilleure gestion des dépendances.

### YOLOv6 (2022) - Développé par Meituan

**Innovations :**
  - Nouveau Backbone, YOLOv6S, optimisé pour les performances en temps réel.
  - Techniques avancées de réduction de la latence.
  - Améliorations dans les méthodes d'augmentation des données et l'optimisation des hyperparamètres.

### YOLOv7 (2022)
**Papier : [YOLOv7: Trainable bag-of-freebies sets new state-of-the-art for real-time object detectors](https://arxiv.org/abs/2207.02696)**

**Innovations :**
  - Intégration de "bag of freebies" pour améliorer la précision sans augmenter le temps d'inférence.
  - Architecture optimisée pour un compromis optimal entre vitesse et précision.
  - Ajout de diverses techniques de régularisation pour améliorer la performance générale.

### YOLOv8 (2023)
**Développé par Ultralytics**

**Innovations :**
  - Encore plus optimisé pour les performances en temps réel et l'intégration mobile.
  - Architecture flexible permettant des ajustements pour divers cas d'utilisation, y compris la détection, la segmentation, et la classification.

### YOLO-World (2024)
**Papier : [YOLO-World: Real-Time Open-Vocabulary Object Detection](https://arxiv.org/pdf/2401.17270)**

**Innovations :**
  - Utilisation d'un transformer encoder pour le texte permettant la détection open-vocabulary.